# Advanced Big Data Analytics - Assignment 3  
# Programming with GPUs using CUDA and pyCUDA

## Setting up  and verifying the environment:

- I followed the instructions on [this link](http://tleyden.github.io/blog/2014/10/25/cuda-6-dot-5-on-aws-gpu-instance-running-ubuntu-14-dot-04/) to first get an Amazon Machine Image with CUDA installed on it.  
- On searching for an AMI with the ID `ami-2cbf3e44`, 2 images show up, one which just has CUDA installed on it and the other with all the Python dependencies required to install pyCUDA bundled in as well. I chose the latter one with all the Python dependencies installed.
- Once the machine was set up, we have to install pyCUDA. To install pyCUDA, I followed the instructions provided on the [pyCUDA Ubuntu Installation Page](https://wiki.tiker.net/PyCuda/Installation/Linux/Ubuntu) to get pyCUDA set up and ready to rumble.  

To install pyCUDA correctly, we run the following commands:  
- First download the newest version of pyCUDA and unpack it
```shell
$ wget https://pypi.python.org/packages/source/p/pycuda/pycuda-2016.1.tar.gz
$ tar zxvf pycuda-2016.1.tar.gz
```
- Then, go into the unpacked directory and set up the configuration file
```shell
$ cd pycuda-2016.1
```
    
    - Since we are using a 64 bit machine with Python 2.7, we have to run the following command to set the configuration correctly
    ```shell
    $ ./configure.py --cuda-root=/usr/local/cuda --cudadrv-lib-dir=/usr/lib/x86_64-linux-gnu --boost-inc-dir=/usr/include --boost-lib-dir=/usr/lib --boost-python-libname=boost_python --boost-thread-libname=boost_thread --no-use-shipped-boost
    ```
    
- Then simply make and install the package
```shell
$ make -j 4
$ sudo python setup.py install
$ sudo pip install .
```

To verify the installation, we run the example of pyCUDA on the landing page of the documentation (`pycuda_docs_hello.py`)

In [ ]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy

from pycuda.compiler import SourceModule
mod = SourceModule("""
__global__ void multiply_them(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}
""")

multiply_them = mod.get_function("multiply_them")

a = numpy.random.randn(400).astype(numpy.float32)
b = numpy.random.randn(400).astype(numpy.float32)

dest = numpy.zeros_like(a)
multiply_them(
        drv.Out(dest), drv.In(a), drv.In(b),
        block=(400,1,1), grid=(1,1))
print dest - a*b

This code simply returns a 2D array of all 0s, as it computes the dot product of the 2 matrices `a` and `b` using the GPU and simply as a `numpy` function, and prints the difference between the 2.  
Seeing all 0s makes it easy to verify that both computations gave the same result.

## Implementing and Running a more complicated example:

The following code is adapted from the [Tiled Matrix Multiplication](https://wiki.tiker.net/PyCuda/Examples/MatrixmulTiled) code on the pyCUDA wiki.  
The code is documented with comments wherever necessary

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import division

""" 
This code is adapted from the example found at : https://wiki.tiker.net/PyCuda/Examples/MatrixmulTiled

The program takes 2 square matrices as numpy arrays, 
and multiplies them together using multiple blocks and shared memory. 
Each thread block is assigned a "tile" of the resulting matrix and is responsible
for generating the elements in that tile.  Each thread in a block computes one element 
of the tile.

We also compute the difference in the result when the calculations are done on
the CPU vs the GPU using the gpuarray module of pycuda
"""

import numpy as np
from numpy import linalg as la
from pycuda import driver, compiler, gpuarray, tools

# -- initialize the device
import pycuda.autoinit

# the C code that will run as a multithreaded process on the GPU
kernel_code_template = """
__global__ void MatrixMulKernel(float *A, float *B, float *C)
{

  const uint wA = %(MATRIX_SIZE)s;
  const uint wB = %(MATRIX_SIZE)s;  
  
  // Block index
  const uint bx = blockIdx.x;
  const uint by = blockIdx.y;

  // Thread index
  const uint tx = threadIdx.x;
  const uint ty = threadIdx.y;

  // Index of the first sub-matrix of A processed by the block
  const uint aBegin = wA * %(BLOCK_SIZE)s * by;
  // Index of the last sub-matrix of A processed by the block
  const uint aEnd = aBegin + wA - 1;
  // Step size used to iterate through the sub-matrices of A
  const uint aStep = %(BLOCK_SIZE)s;

  // Index of the first sub-matrix of B processed by the block
  const uint bBegin = %(BLOCK_SIZE)s * bx;
  // Step size used to iterate through the sub-matrices of B
  const uint bStep = %(BLOCK_SIZE)s * wB;

  // The element of the block sub-matrix that is computed
  // by the thread
  float Csub = 0;
  // Loop over all the sub-matrices of A and B required to
  // compute the block sub-matrix
  for (int a = aBegin, b = bBegin;
       a <= aEnd;
       a += aStep, b += bStep) 
    {
      // Shared memory for the sub-matrix of A
      __shared__ float As[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];
      // Shared memory for the sub-matrix of B
      __shared__ float Bs[%(BLOCK_SIZE)s][%(BLOCK_SIZE)s];

      // Load the matrices from global memory to shared memory
      // each thread loads one element of each matrix
      As[ty][tx] = A[a + wA * ty + tx];
      Bs[ty][tx] = B[b + wB * ty + tx];
      // Synchronize to make sure the matrices are loaded
      __syncthreads();

      // Multiply the two matrices together;
      // each thread computes one element
      // of the block sub-matrix
      for (int k = 0; k < %(BLOCK_SIZE)s; ++k)
        Csub += As[ty][k] * Bs[k][tx];

      // Synchronize to make sure that the preceding
      // computation is done before loading two new
      // sub-matrices of A and B in the next iteration
      __syncthreads();
    }

  // Write the block sub-matrix to global memory;
  // each thread writes one element
  const uint c = wB * %(BLOCK_SIZE)s * by + %(BLOCK_SIZE)s * bx;
  C[c + wB * ty + tx] = Csub;
}
"""

# define the (square) matrix size
# Since we want to work with matrices of size 50 MB or so, we need to do some calculations
# to determine what the size of the input sqaure matrices will be
'''
1 float = 8 bits
50 mb = 50 * 1024 * 1024
Total number of floats required in matrices = 50 * 1024 * 1024 / 8

Using this, the size of the matrix = sqrt(50 * 1024 * 1024 / 8) = 2560
Hence, to manipulate matrices of size 50 mb each, 
we need the total number of elements in the matrix to be 2560 x 2560
therefore, set the size of the matrix as 2560
'''
MATRIX_SIZE = 2560 

# define size of blocks and tiles sub-matrix 
# (we assume that the block size is same as tile size)
# pycuda will assign a particular number of threads to each block
# checking the output of dump_properties.py file, we see that we can have at most
# 1024 threads per block
# as long as the total number of elements in a tile are less than 1024, pycuda can execute this code without any problems 
TILE_SIZE = 32 
BLOCK_SIZE = TILE_SIZE

# create two random square matrices, each of size 50 mb
a_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)
b_cpu = np.random.randn(MATRIX_SIZE, MATRIX_SIZE).astype(np.float32)

# compute reference on the CPU to verify GPU computation
c_cpu = np.dot(a_cpu, b_cpu)

# transfer host (CPU) memory to device (GPU) memory 
a_gpu = gpuarray.to_gpu(a_cpu) 
b_gpu = gpuarray.to_gpu(b_cpu)

# create empty gpu array for the result (C = A * B)
c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)

# get the kernel code from the template 
# by specifying the constants MATRIX_SIZE and BLOCK_SIZE
kernel_code = kernel_code_template % { 
    'MATRIX_SIZE': MATRIX_SIZE,
    'BLOCK_SIZE': BLOCK_SIZE,
    }

# compile the kernel code
mod = compiler.SourceModule(kernel_code)

# get the kernel function from the compiled module
matrixmul = mod.get_function("MatrixMulKernel")

# call the kernel on the Graphics card
matrixmul(
    # inputs
    a_gpu, b_gpu, 
    # output
    c_gpu, 
    # grid of multiple blocks
    grid = (MATRIX_SIZE // TILE_SIZE, MATRIX_SIZE // TILE_SIZE),
    # block of multiple threads
    block = (TILE_SIZE, TILE_SIZE, 1), 
    )

# print the results
print "-" * 80
print "Matrix A (GPU):"
print a_gpu.get()

print "-" * 80
print "Matrix B (GPU):"
print b_gpu.get()

print "-" * 80
print "Matrix C (GPU):"
print c_gpu.get()

print "-" * 80
print "CPU-GPU difference:"
print c_cpu - c_gpu.get()
print "L2 norm:", la.norm(c_cpu - c_gpu.get())
np.allclose(c_cpu, c_gpu.get())


### Discussion : 
There are some important things to note here (also mentioned in code comments):
- Since we want to deal with data that is at least 50 MB in size, we first need to calculate how many elements we will need in each matrix to make it require 50 MB of memory. To find the exact number, we need to do some calculations:

```
1 float = 8 bits
50 mb = 50 * 1024 * 1024
Total number of floats required in matrices = 50 * 1024 * 1024 / 8

Using this, the size of the matrix = sqrt(50 * 1024 * 1024 / 8) = 2560
Hence, to manipulate matrices of size 50 mb each, 
we need the total number of elements in the matrix to be 2560 x 2560
which means that we have to set the MATRIX_SIZE parameter as 2560
```

- The tile size cannot be arbitrarily large.  
    Each thread block is assigned a "tile" of the resulting matrix and is responsible
    for generating the elements in that tile.  Each thread in a block computes one element 
    of the tile. The question then is how to choose a good tile size.
    
In the pyCUDA examples directory in the unpacked `pycuda-2016.1` folder, there is a file called `dump_properties.py`, which simply prints out all of the default and user defined properties of pyCUDA and the underlying CUDA installation.
In this dump, we see that the `MAX_THREAD_BLOCK_SIZE` is set to 1024. i.e., there can be at most 1024 separate threads for each block.  
Each thread block deals with 1 tile, and each thread in the block is responsible for computing the value of one element in the tile.  
The `TILE_SIZE` parameter in the code is responsible for making a thread block of size `TILE_SIZE x TILE_SIZE`.  
Therefore, it can be set to a maximum of `sqrt(1024) = 32`  
Since we are dealing with very very large matrices, it makes sense to use mulithreading in total force and set the tile size to 32.


On running the above program, we get the following output in the shell, which prints the 2 matrices `a` and `b` and the result of the computation `a * b`

In [1]:
cat output.py

--------------------------------------------------------------------------------
Matrix A (GPU):
[[ 0.01775491  0.70539278 -0.87263292 ...,  0.2844947  -0.21808594
   1.21563804]
 [-0.6686821   0.41078544  1.21795642 ...,  0.1043731   0.07380684
  -0.73897499]
 [-0.94774461  2.66338634 -0.33823362 ..., -0.49412024  1.73328018
  -1.50323164]
 ..., 
 [-1.79621673  1.41672575  1.84087658 ...,  1.06714141  0.70821875
   2.42188454]
 [-0.21768838  0.40100154 -1.41286051 ...,  0.85705167 -0.44164455
   0.40237099]
 [-0.99681801 -0.03101943 -0.63504004 ..., -0.44020164 -0.47245443
  -0.05559199]]
--------------------------------------------------------------------------------
Matrix B (GPU):
[[ 0.87681538 -1.21908629  1.6373837  ..., -0.11167652  0.09793786
  -0.60445577]
 [ 2.1978476   0.8985244   0.28708866 ...,  0.2088144  -0.16963698
  -0.74097604]
 [-0.61910552  0.11249758  2.48630381 ...,  0.48025823  0.1537658
  -0.41130912]
 ..., 
 [ 0.36054409  1.8271873   1.74100745 ..., -0.94321477

From the output, we can see that the CPU - GPU difference is fairly low for the individual terms (order of 10e-5 or 10e-6).
MSE of CPU-GPU difference  = L2 Norm = 0.11035  
This is a very small value given that we are taking the L2 norm over 2560 * 2560 elements